<a href="https://colab.research.google.com/github/IFuentesSR/SAMSARA_UAI/blob/main/getting_inputs_using_CCDC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Codes necessary to obtain inputs for secondary classification

In [1]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize(project='your_project') #use your credentials

#### Section to get the CCDC covariates (change dates and magnitudes)

In [2]:
def mask(img):
    qua = ee.Image(img).select('QA_PIXEL')
    props = img.propertyNames()
    dilated = qua.bitwiseAnd(2).eq(0)
    cirrus = qua.bitwiseAnd(4).eq(0)
    clouds = qua.bitwiseAnd(8).eq(0)
    shadows = qua.bitwiseAnd(16).eq(0)
    return img.updateMask(dilated).updateMask(cirrus).updateMask(clouds).updateMask(shadows).multiply(0.0000275).add(-0.2).copyProperties(img, props)

def ndvil8(img):
    props = img.propertyNames()
    ix = img.normalizedDifference(['SR_B5', 'SR_B4'])
    return img.addBands(ix.rename('ndvi')).copyProperties(img, props)


def ndvil57(img):
    props = img.propertyNames()
    ix = img.normalizedDifference(['SR_B4', 'SR_B3'])
    return img.addBands(ix.rename('ndvi')).copyProperties(img, props)


def get_date(x):
    return ee.Date(x).format('YYYY-MM-dd')

In [3]:
"""functions taken from users/wiell/temporalSegmentation:temporalSegmentation"""

def count(segmentsImage):
    return segmentsImage.select(0).arrayLength(0)


def getSegmentIndexes(segmentsImage):
    return segmentsImage.select(0).Not().Not().arrayAccum(0, ee.Reducer.sum()).subtract(1)


def bandNames1D(segmentsImage):
    return segmentsImage.bandNames().filter(ee.Filter.stringEndsWith('item', '_coefs').Not())


def bandNames2D(segmentsImage):
    return segmentsImage.bandNames().filter(ee.Filter.stringEndsWith('item', '_coefs'))


def updateImageMask(segmentsImage):
    return segmentsImage.addBands(segmentsImage.select(bandNames1D(segmentsImage)).unmask(ee.Array([], ee.PixelType.double())), None, True).addBands(segmentsImage.select(bandNames2D(segmentsImage)).unmask(ee.Array([[]], ee.PixelType.double())), None, True).mask(segmentsImage.select(0).arrayLength(0).unmask(0))


def getSegmentImage(segmentIndex, segmentsImage):
    mask = getSegmentIndexes(segmentsImage).eq(segmentIndex.unmask(-1))
    image1D = segmentsImage.select(bandNames1D(segmentsImage)).arrayMask(mask)
    image1D = image1D.mask(image1D.select(0).arrayLength(0).unmask(0)).arrayProject([0]).arrayGet([0])
    image2D = segmentsImage.select(bandNames2D(segmentsImage)).arrayMask(mask.toArray(1).unmask(ee.Array([[]], ee.PixelType.double())))
    image2D = image2D.mask(image2D.select(0).arrayLength(0).unmask(0)).arrayProject([1])
    return image1D.addBands(image2D)


def toCollection(segmentsImage, maxSegments):
    segmentCount = count(segmentsImage)
    def inner(i, acc):
        image = ee.Image([0]).set('imageIndex', i)
        return ee.ImageCollection(acc).merge(ee.ImageCollection([image]))
    def inner2(image):
        imageIndex = ee.Image(ee.Number(image.get('imageIndex')))
        segmentIndex = segmentCount.subtract(1).min(imageIndex)
        return getSegmentImage(segmentIndex, segmentsImage).updateMask(imageIndex.lt(segmentCount))

    imageCollection = ee.ImageCollection(ee.List.sequence(0, ee.Number(maxSegments).subtract(1)).iterate(inner, ee.ImageCollection([]))).map(inner2)
    return imageCollection


def ccdcProcessing(segmentsImage, maxSegments):
    segmentsImage = updateImageMask(segmentsImage)
    segmentsImage = segmentsImage.addBands(segmentsImage.select('.*_coefs').arrayPad([0, 8]), None, True)
    return toCollection(segmentsImage, maxSegments)


def iteration(image, acc):
    return ee.Image(acc).addBands(image)

Different tiles in the study region. Consider to iterate using them to cover the entire area of study.

In [ ]:
tiles = ee.List(['001083', '001084', '001085', '233083', '233084', '233085'])
ix = 0

In [ ]:
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filter(ee.Filter.stringContains('system:index', tiles.get(ix))).map(mask).map(ndvil8).filterDate('2000-01-01', '2023-01-01')
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").filter(ee.Filter.stringContains('system:index', tiles.get(ix))).map(mask).map(ndvil8).filterDate('2000-01-01', '2023-01-01')
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2").filter(ee.Filter.stringContains('system:index', tiles.get(ix))).map(mask).map(ndvil57).filterDate('2000-01-01', '2023-01-01')
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").filter(ee.Filter.stringContains('system:index', tiles.get(ix))).map(mask).map(ndvil57).filterDate('2000-01-01', '2023-01-01')

In [ ]:
landsat = l9.merge(l8).merge(l7).merge(l5).select('ndvi').sort('system:time_start')
geo = ee.Geometry.Polygon(ee.Geometry(l9.first().get('system:footprint')).coordinates())
ccdc = ee.Algorithms.TemporalSegmentation.Ccdc(collection=landsat,
                                               breakpointBands=['ndvi'],
                                               tmaskBands=None,
                                               minObservations=5,
                                               chiSquareProbability=0.9)

In [ ]:
maxSegments = 5
band = 'tStart' # 'ndvi_magnitude'

collection = ccdcProcessing(ccdc, maxSegments)
allCcdcBands = ee.Image(collection.iterate(iteration, ee.Image())).slice(1)
singleCcdcBands = ee.Image(collection.select(band).iterate(iteration, ee.Image())).slice(1)

In [ ]:
ee.batch.Export.image.toAsset(image=singleCcdcBands,
                              region=geo,
                              scale=30,
                              maxPixels=1e13,
                              description='ccdc_ex',
                              assetId='users/ignisfausto/ccdc_test_ex_tStart').start()

#### Section to get the radar vegetation index (RVI) covariates

$RVI = \frac{4\sigma^0_{VH}}{\sigma^0_{VV}+\sigma^0_{VH}}$

In [ ]:
### Functions to process SAR data

def toNatural(img):
  img = ee.Image(img)
  return ee.Image(10.0).pow(img.select(['VV', 'VH']).divide(10.0))


def toDB(img):
  img = ee.Image(img)
  return ee.Image(img).log10().multiply(10.0).copyProperties(img, ['system:time_start', 'date'])


def edgeRemoval(img):
  img = ee.Image(img)
  return img.updateMask(img.gt(-30))


def rviFunction(img):
    # #https://www.tandfonline.com/doi/full/10.1080/22797254.2021.2018667
    props = img.propertyNames()
    rvi = img.select('VH').multiply(4).divide(img.select('VH').add(img.select('VV')))
    return rvi.rename('rvi').copyProperties(img, props)


def set_datex(img):
    return img.set('date', ee.Date(ee.Image(img).get('system:time_start')).format('YYYY-MM-dd'))


def gaussianFilter(img):
  properties = img.propertyNames()
  img = ee.Image(edgeRemoval(img))
  return img.convolve(ee.Kernel.gaussian(3, 3)).copyProperties(img, properties)

In [ ]:
geometry = ee.Geometry.Polygon([[[-70.92753862061205, -32.977644498424056],
                                 [-70.92839692749682, -32.97620444855654],
                                 [-70.92977021851245, -32.97591643576499],
                                 [-70.92925523438159, -32.9779325055795]]])

In [ ]:
S1 = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
S1 = S1.filterBounds(geometry).filterDate('2016-01-01', '2023-01-01')
S1 = S1.filter(ee.Filter.eq('instrumentMode', 'IW'))
S1 = S1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
S1 = S1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
S1 = S1.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
S1 = S1.map(gaussianFilter)

In [ ]:
SRI = S1.map(rviFunction).map(set_datex).sort('system:time_start')
agg = SRI.reduce(ee.Reducer.percentile([25, 75]))
agg = agg.addBands(agg.select(1).subtract(agg.select(0)).rename('iqr'))

In [ ]:
ee.batch.Export.image.toDrive(image=agg,
                              description='any',
                              scale=30,
                              maxPixels=1e13).start()

['rvi_p25', 'rvi_p75', 'iqr']


#### Section to get inputs derived from CCDC temporal segmentation using neighborhood and textural analysis

In [9]:
def to_year (img):
    year2000 = 730485
    year = img.subtract(year2000).divide(365)
    return ee.Image(2000).add(ee.Image(year))


def to_day(img):
    year = img.subtract(2016).multiply(365)
    return year


In [25]:
'''import the saved ccdc outputs that have been created (change magnitudes and dates)'''

mgs = ee.Image('users/ignisfausto/ccdc_magnitude_001083_09')
bks = ee.Image('users/ignisfausto/ccdc_test_ex_tStart')

In [27]:
'''Neighborhood statistics'''

mask = bks.gte(2016)
mask2 = mgs.lt(0)
mask2 = ee.Image(0).addBands(mask2.select([0,1,2,3]))

bks = bks.updateMask(mask).updateMask(mask2)
bks = bks.select([1,2,3,4])
mgs = mgs.updateMask(mask.select([1,2,3,4]).addBands(ee.Image(0))).updateMask(mask2.select([1,2,3,4]).addBands(ee.Image(0)))

bks2 = bks.reduce(ee.Reducer.firstNonNull()).rename('ccdc_bks')
mgs2 = mgs.reduce(ee.Reducer.firstNonNull()).rename('ccdc_mgs')

ccdc_dv = ee.Image(bks2).addBands(mgs2).reduceNeighborhood('stdDev', ee.Kernel.square(50))
ee.batch.Export.image.toDrive(image=ccdc_dv,
                              description='any',
                              region=geo,
                              scale=30,
                              maxPixels=1e13).start()


In [23]:
'''GLCM inputs'''

focal = mgs2.focal_mean(7)
mgs3 = mgs2.where(mgs2.subtract(focal).abs().gt(0.3), focal)
glcm = mgs3.multiply(1000).toInt().glcmTexture(7, ee.Kernel.square(7))
bands = ['ccdc_mgs_asm',
         'ccdc_mgs_contrast',
         'ccdc_mgs_idm',
         'ccdc_mgs_ent',
         'ccdc_mgs_savg']
ee.batch.Export.image.toDrive(image=glcm.select(bands),
                              description='any',
                              region=geo,
                              scale=30,
                              maxPixels=1e13).start()